# Section 3：ExcelのデータをPythonで読み込んでみよう

## JupyterLabからライブラリをインストールする

In [ ]:
# Windowsの場合
!pip install pandas
!pip install openpyxl

In [ ]:
# Macの場合
!pip3 install pandas
!pip3 install openpyxl

In [ ]:
import pandas as pd
import openpyxl
from glob import glob
import datetime

## Excelファイルを読み込む

In [ ]:
file_paths = glob('src/20220131/*.xlsx')
file_paths

In [ ]:
file_path = file_paths[0]
file_path

In [ ]:
# Excelファイルの読み込み（wb: workbookの略）
wb = openpyxl.load_workbook(file_path)

In [ ]:
wb

In [ ]:
# シート名の取得
wb.sheetnames

## シートの読み込み

In [ ]:
# シートの取得　（ws: worksheetの略）
ws = wb['timesheet']
ws

## セルのオブジェクトと値を読み込む

In [ ]:
ws['C3']

In [ ]:
ws['C3'].value

In [ ]:
# data_only=Trueを指定
wb = openpyxl.load_workbook(file_path, data_only=True)
# シートの取得（ws: worksheetの略）
ws = wb['timesheet']
# C3セルの値を取得
ws['C3'].value

# Section 4：Excelのシートから必要な情報を読み込もう

In [ ]:
# 氏名
name = ws['C3'].value
name

In [ ]:
ws['B6:F12']

In [ ]:
type(ws['B6:F12'])

In [ ]:
ws['B6:F12'][0]

In [ ]:
ws['B6:F12'][0][0]

In [ ]:
for row in ws['B6:F12']:
    for cell in row:
        print(cell)

In [ ]:
datum = []
for row in ws['B6:F12']:
    # 各行の値を格納しておくリストrow_datumを用意
    row_datum = []
    for cell in row:
        # セルの値をrow_datumに格納
        row_datum.append(cell.value)
    # 1行分の値が格納されたrow_datumをdatumに格納
    datum.append(row_datum)
datum

In [ ]:
pd.DataFrame(datum)

In [ ]:
datum[0][4]

In [ ]:
datum[0][4] == datetime.datetime(1899, 12, 30, 0, 0)

In [ ]:
datum = []
for row in ws['B6:F12']:
    row_datum = []
    for cell in row:
        basis_date = datetime.datetime(1899, 12, 30, 0, 0)
        if cell.value == basis_date:
            # セルの値が0:00だった場合、Noneを追加
            row_datum.append(None)
        else:
            # セルの値が0:00ではなかった場合、セルの値をそのまま追加
            row_datum.append(cell.value)
    datum.append(row_datum)
pd.DataFrame(datum)

# Section 5：集約用のExcelファイルを準備しよう

## 集約用のExcelファイルを作成する

In [ ]:
# 新規Workbookオブジェクトを作成
wb_members = openpyxl.Workbook()
# 従業員集約.xlsxというファイル名でWorkbookを保存
wb_members.save('従業員集約.xlsx')

In [ ]:
wb_members.create_sheet(title=name)

In [ ]:
wb_members.save('従業員集約.xlsx')

In [ ]:
# 不要なシートを削除
wb_members.remove(wb_members['Sheet'])
# 削除を反映するために保存
wb_members.save('従業員集約.xlsx')

## カラム（列）の名前を設定しよう

In [ ]:
# シートを指定
ws_members = wb_members[name]
# 各セルに値を入力
ws_members['A1'].value = '日付'
ws_members['B1'].value = '出勤時間'
ws_members['C1'].value = '退勤時間'
ws_members['D1'].value = '休憩時間'
ws_members['E1'].value = '作業時間'

In [ ]:
wb_members.save('従業員集約.xlsx')

In [ ]:
ws_members.column_dimensions['A'].width = 15
ws_members.column_dimensions['B'].width = 15
ws_members.column_dimensions['C'].width = 15
ws_members.column_dimensions['D'].width = 15
ws_members.column_dimensions['E'].width = 15

In [ ]:
wb_members.save('従業員集約.xlsx')

## 勤怠情報を転記しよう

In [ ]:
max_row = ws_members.max_row
max_row

In [ ]:
# row_numに1〜7の値が順番に入る
for row_num in range(1, 8):
    # column_numに1〜5の値が順番に入る
    for column_num in range(1, 6):
        # リストは0番目から始まるので-1する
        ws_members.cell(
            row=max_row+row_num, column=column_num
        ).value = datum[row_num-1][column_num-1]

In [ ]:
wb_members.save('従業員集約.xlsx')

### 表示形式の設定

In [ ]:
# 日付形式
ws_members.cell(
    row=max_row+row_num, column=column_num
).number_format = 'yy/mm/dd'
# 時刻形式
ws_members.cell(
    row=max_row+row_num, column=column_num
).number_format = 'h:mm'

In [ ]:
for row_num in range(1, 8):
    for column_num in range(1,6):
        # 1列目は日付形式、それ以外は時刻形式、という条件分岐
        if column_num == 1:
            ws_members.cell(
                row=max_row+row_num, column=column_num
            ).number_format = 'yyyy/m/d'
        else:
            ws_members.cell(
                row=max_row+row_num, column=column_num
            ).number_format = 'h:mm'
        ws_members.cell(
            row=max_row+row_num, column=column_num
        ).value = datum[row_num-1][column_num-1]
wb_members.save('従業員集約.xlsx')

## まとめたコード

In [ ]:
import openpyxl
from glob import glob
import datetime

# 　＜---必要な情報の取得---＞
file_paths = glob('src/20220131/*.xlsx')
file_path = file_paths[0]
wb = openpyxl.load_workbook(file_path, data_only=True)
ws = wb['timesheet']

name = ws['C3'].value
datum = []
for row in ws['B6:F12']:
    row_datum = []
    for cell in row:
        basis_date = datetime.datetime(1899, 12, 30, 0, 0)
        if cell.value == basis_date:
            row_datum.append(None)            
        else:
            row_datum.append(cell.value)
    datum.append(row_datum)

# 　＜---勤怠情報の転記---＞    
# 作成ずみの「従業員集約 .xlsx」を読み込む形に変更
wb_members = openpyxl.load_workbook(
    '従業員集約.xlsx', data_only=True)
# シートが作成ずみかどうかで条件分岐
if name not in wb_members.sheetnames:
    # シートが作成されていなかったら作成し、カラム名、カラム幅を設定
    wb_members.create_sheet(title=name)
    ws_members = wb_members[name]

    ws_members['A1'].value = '日付'
    ws_members['B1'].value = '出勤時間'
    ws_members['C1'].value = '退勤時間'
    ws_members['D1'].value = '休憩時間'
    ws_members['E1'].value = '作業時間'

    ws_members.column_dimensions['A'].width = 15
    ws_members.column_dimensions['B'].width = 15
    ws_members.column_dimensions['C'].width = 15    
    ws_members.column_dimensions['D'].width = 15    
    ws_members.column_dimensions['E'].width = 15    

else:
    # シートが作成されていたら、そのシートを読み込む
    ws_members = wb_members[name]

max_row = ws_members.max_row

for row_num in range(1, 8):
    for column_num in range(1,6):
        if column_num == 1:
            ws_members.cell(
                row=max_row+row_num, column=column_num
            ).number_format = 'yyyy/m/d'
        else:
            ws_members.cell(
                row=max_row+row_num, column=column_num
            ).number_format = 'h:mm'
        ws_members.cell(
            row=max_row+row_num, column=column_num
        ).value = datum[row_num-1][column_num-1]

wb_members.save('従業員集約.xlsx')

# Section 6：全員の勤怠情報を1つのファイルに集約しよう

## まとめて処理する用のプログラム

In [ ]:
import openpyxl
from glob import glob
import datetime

# 　＜---必要な情報の取得---＞
file_paths = glob('src/20220131/*.xlsx')
# 変更部分：従業員集約.xlsxの読み込みを上部に移動
wb_members = openpyxl.load_workbook(
    '従業員集約.xlsx', data_only=True)
for file_path in file_paths:
    wb = openpyxl.load_workbook(file_path, data_only=True)
    ws = wb['timesheet']

    name = ws['C3'].value
    datum = []
    for row in ws['B6:F12']:
        row_datum = []
        for cell in row:
            basis_date = datetime.datetime(1899, 12, 30, 0, 0)
            if cell.value == basis_date:
                row_datum.append(None)            
            else:
                row_datum.append(cell.value)
        datum.append(row_datum)

    # 　＜---勤怠情報の転記---＞    
    # シートが作成ずみかどうかで条件分岐
    if name not in wb_members.sheetnames:
        # シートが作成されていなかったら作成し、カラム名、カラム幅を設定
        wb_members.create_sheet(title=name)
        ws_members = wb_members[name]

        ws_members['A1'].value = '日付'
        ws_members['B1'].value = '出勤時間'
        ws_members['C1'].value = '退勤時間'
        ws_members['D1'].value = '休憩時間'
        ws_members['E1'].value = '作業時間'

        ws_members.column_dimensions['A'].width = 15
        ws_members.column_dimensions['B'].width = 15
        ws_members.column_dimensions['C'].width = 15    
        ws_members.column_dimensions['D'].width = 15    
        ws_members.column_dimensions['E'].width = 15    

    else:
        # シートが作成されていたら、そのシートを読み込む
        ws_members = wb_members[name]

    max_row = ws_members.max_row

    for row_num in range(1, 8):
        for column_num in range(1,6):
            if column_num == 1:
                ws_members.cell(
                    row=max_row+row_num, column=column_num
                ).number_format = 'yyyy/m/d'
            else:
                ws_members.cell(
                    row=max_row+row_num, column=column_num
                ).number_format = 'h:mm'
            ws_members.cell(
                row=max_row+row_num, column=column_num
            ).value = datum[row_num-1][column_num-1]

    wb_members.save('従業員集約.xlsx')

# Section 7：他の週次フォルダも処理できるようにしよう

### 【注意】
以下のプログラムは、ターミナルから実行する**Pythonファイル**として調整しています。<br>
**ノートブック上ではエラーが発生**し、正常に実行できません。

In [ ]:
import openpyxl
from glob import glob
import datetime

# 追加
import sys
args = sys.argv

# 　＜---必要な情報の取得---＞
file_paths = glob(f'{args[1]}/*.xlsx')
# 変更部分：従業員集約.xlsxの読み込みを上部に移動
wb_members = openpyxl.load_workbook(
    '従業員集約.xlsx', data_only=True)
for file_path in file_paths:
    wb = openpyxl.load_workbook(file_path, data_only=True)
    ws = wb['timesheet']

    name = ws['C3'].value
    datum = []
    for row in ws['B6:F12']:
        row_datum = []
        for cell in row:
            basis_date = datetime.datetime(1899, 12, 30, 0, 0)
            if cell.value == basis_date:
                row_datum.append(None)            
            else:
                row_datum.append(cell.value)
        datum.append(row_datum)

    # 　＜---勤怠情報の転記---＞    
    # シートが作成ずみかどうかで条件分岐
    if name not in wb_members.sheetnames:
        # シートが作成されていなかったら作成し、カラム名、カラム幅を設定
        wb_members.create_sheet(title=name)
        ws_members = wb_members[name]

        ws_members['A1'].value = '日付'
        ws_members['B1'].value = '出勤時間'
        ws_members['C1'].value = '退勤時間'
        ws_members['D1'].value = '休憩時間'
        ws_members['E1'].value = '作業時間'

        ws_members.column_dimensions['A'].width = 15
        ws_members.column_dimensions['B'].width = 15
        ws_members.column_dimensions['C'].width = 15    
        ws_members.column_dimensions['D'].width = 15    
        ws_members.column_dimensions['E'].width = 15    

    else:
        # シートが作成されていたら、そのシートを読み込む
        ws_members = wb_members[name]

    max_row = ws_members.max_row

    for row_num in range(1, 8):
        for column_num in range(1,6):
            if column_num == 1:
                ws_members.cell(
                    row=max_row+row_num, column=column_num
                ).number_format = 'yyyy/m/d'
            else:
                ws_members.cell(
                    row=max_row+row_num, column=column_num
                ).number_format = 'h:mm'
            ws_members.cell(
                row=max_row+row_num, column=column_num
            ).value = datum[row_num-1][column_num-1]

    wb_members.save('従業員集約.xlsx')

# 追加
print('集約完了')